In [1]:
import mysql.connector
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import COMMASPACE
from email import encoders


In [3]:
# Connect to MySQL server
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="users"
)

In [ ]:
# Create cursor to execute queries
mycursor = mydb.cursor()

# Query 'users' table to get email and ID
mycursor.execute("SELECT email, ID FROM users")

# Fetch all rows of the query result
users = mycursor.fetchall()

# Loop through each user
for user in users:
    # Query 'visited_sites' table to get VISITED value of each site_name for the user
    mycursor.execute("SELECT site_name, VISITED FROM visited_sites WHERE user_ID = %s", (user[1],))
    visited_sites = mycursor.fetchall()

    # Create a list of not visited sites
    not_visited_sites = []
    for site in visited_sites:
        if not site[1]:
            not_visited_sites.append(site[0])
    
    # If the user has not visited any sites, send email with list of sites and a picture of each site
    if not_visited_sites:
        # Format email content
        email_content = f"Dear {user[0]},\n\nYou have not yet visited:\n"
        for site_name in not_visited_sites:
            email_content += f"- {site_name}\n"
        email_content += "\nPlease take the opportunity to visit these sites. Otherwise, please enjoy a picture of them:\n"

        # Send email with pictures of sites
        for site_name in not_visited_sites:
            email_content += f"{site_name}.jpg\n"

        # Send email using a SMTP server or service like Gmail, Mailgun, or SendGrid

In [ ]:
smtp_server = 'smtp.gmail.com'
smtp_port = 587
smtp_username = 'yourgmailusername@gmail.com'
smtp_password = 'yourgmailpassword'

In [ ]:
for user in users:
# Query 'visited_sites' table to get VISITED value of each site_name for the user
    mycursor.execute("SELECT site_name, VISITED FROM visited_sites WHERE user_ID = %s", (user[1],))
    visited_sites = mycursor.fetchall()

In [ ]:
# Create a list of not visited sites
not_visited_sites = []
for site in visited_sites:
    if not site[1]:
        not_visited_sites.append(site[0])

# If the user has not visited any sites, send email with list of sites and a picture of each site
if not_visited_sites:
    # Format email content
    email_content = f"Dear {user[0]},\n\nYou have not yet visited:\n"
    for site_name in not_visited_sites:
        email_content += f"- {site_name}\n"
    email_content += "\nPlease take the opportunity to visit these sites. Otherwise, please enjoy a picture of them:\n"

    # Create a MIME message object and add sender, recipient and subject
    msg = MIMEMultipart()
    msg['From'] = smtp_username
    msg['To'] = user[0]
    msg['Subject'] = 'Sites to visit'
    
    # Add text content to the message
    text = MIMEText(email_content)
    msg.attach(text)
    
    # Add pictures of sites to the message
    for site_name in not_visited_sites:
        filepath = f'../Heritage/{site_name}.jpg'
        with open(filepath, 'rb') as f:
            img = MIMEBase('image', 'jpg', name=site_name+'.jpg')
            img.set_payload(f.read())
            encoders.encode_base64(img)
            img.add_header('Content-Disposition', 'attachment', filename=site_name+'.jpg')
            msg.attach(img)
    
    # Send email using SMTP server
    try:
        smtp = smtplib.SMTP(smtp_server, smtp_port)
        smtp.starttls()
        smtp.login(smtp_username, smtp_password)
        smtp.sendmail(smtp_username, user[0], msg.as_string())
        smtp.quit()
        print(f"Email sent to {user[0]}")
    except Exception as e:
        print(f"Email failed to send to {user[0]}. Error message: {str(e)}")
